In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

In [ ]:
matplotlib.rcParams.update({'font.size': 18,
                            'lines.linewidth' : 3,
                           'figure.figsize' : [15, 5],
                           'lines.markersize': 10})
pd.options.mode.chained_assignment = None

# Previous session
We had a teaser of what to expect when doing Machine Learning. By now you should know:
1. How to use `scikit-learn`
    - **predictors**: `fit(X)`, `predict(X)`
    - **transformers**: `fit(X)`, `transform(X)`, `fit_transform(X)`
1. What metrics to use for Supervised classification learning    
1. Be able to build your first `Decision Tree` binary classifier using the *DieTanic* dataset

# Model Selection
We have defined machine learning as the process of optimizing the cost function of the problem by tweaking the parameters of a model. We have defined a model as some map that uniquely maps input (features) to output (labels or target values). We've left this definition very vague, because there are many popular models used for machine learning. We have already looked at decision trees and now we will investigate how to choose the optimal hyperparameters.

In [ ]:
df_titanic = pd.read_csv('./00_data/titanic_X.csv')
df_titanic_target = pd.read_csv('./00_data/titanic_y.csv', squeeze=True,header=None)

df_titanic.head()

In [ ]:
df_titanic_target.head()

In [ ]:
X = np.array(df_titanic)
y = np.array(df_titanic_target)

df_titanic['Survived'] = df_titanic_target

# Exercise 1:
1. Load **./00_data/dietanic.csv**
1. Identify the feature matrix `X` and the labels `y`.
2. Create training and testing dataset: `X_train, y_train, X_test, y_test` of test size 20% with seed 55

# *Recap*: Decision Tree

In [ ]:
import graphviz
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# train decision tree
dtc = DecisionTreeClassifier(max_depth=2,)
y_pred = dtc.fit(X,y).predict(X)

# visual tree
graphviz.Source(export_graphviz(dtc, 
                                out_file=None,
                                feature_names=np.array(df_titanic.drop('Survived', axis=1).columns.tolist()),
                                class_names=np.array(['0','1'])))

## By allowing for more branching, does this make our model better or worse?¶

In [ ]:
from sklearn.metrics import f1_score
print('f1-Score : {}'.format(f1_score(y, y_pred)))

In [ ]:
max_depths = range(1,40)
f1Score = []
for md in max_depths:
    dtc = DecisionTreeClassifier(max_depth=md)
    y_pred = dtc.fit(X,y).predict(X)
    f1Score.append(f1_score(y, y_pred))
    
plt.plot(max_depths, f1Score, label = 'f1-Score')
plt.xlabel('Maximum Depth Hyperparameter')
plt.ylabel('Metric')
plt.legend();

We reach a conflict: the model keeps getting better and better the deeper we go with **max_depth** and beyond 18, we are suffering from _overfitting_. The model does not seem to improve and at 24 max_depth, the precision actually decreases as this now starts to follow noise in the data. To detect overfitting, we need to see how our model generalizes to new data. We can do this artificially by withholding part of our data set during the training step, and then using it to test the model.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_titanic.drop(['Survived'], axis=1),
                                                   df_titanic['Survived'],
                                                   test_size = 0.1)
f1Score_test = []
for md in max_depths:
    dtc = DecisionTreeClassifier(max_depth=md)
    y_pred = dtc.fit(X_train,y_train).predict(X_test)
    f1Score_test.append(f1_score(y_test, y_pred))
    
plt.figure(figsize=(20,10))
plt.plot(max_depths, f1Score, label = 'f1-Score')
plt.plot(max_depths, f1Score_test, '--', label = 'f1-Score test')
plt.xlabel('Maximum Depth Hyperparameter')
plt.ylabel('Metric')
plt.legend();

The testing error confirms our suspicion. As the model becomes more complex, it improves up to a point, and then it loses generalizability. **The testing metrics actually start decreasing!**

# Exercise 2:
1. Train a decision tree classifier with hyperparameter **max_depth = 3** and random_state=55
1. Evaluate the f1-score on the test data
1. Train a decision tree classifier with hyperparameter **max_depth = 15**, random_state=55 and evaluate the f1-score
1. Train a decision tree classifier with hyperparameter **max_depth = 40**, random_state=55 and evaluate the f1-score

# Hyperparameters: tuning and cross-validation

DTC has multiple hyper-parameters:
- `max_depth`,
- `min_samples_split`
- [DTC documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

**Which is optimal?**

In [ ]:
max_depth = [5,10,15,20,25,30]
min_samples_split = [25, 50, 75, 100, 125, 150, 175, 200]
import itertools

combns = [ii  for ii in itertools.product(max_depth, min_samples_split)] 

combns_x = [x[0] for x in combns]
combns_y = [x[1] for x in combns]
plt.scatter(combns_x, combns_y)
plt.xlabel('max_depth')
plt.xticks(max_depth)
plt.ylabel('min_samples_split')
plt.yticks(min_samples_split)
plt.show()

Since changing hyperparameters changes the structure of the model, we should think of choosing hyperparameters as part of model selection. `Scikit-learn` provides a useful tool for comparing different hyperparameter values, `GridSearchCV`. There are two ideas behind `GridSearchCV`: first we will split up the data into a training and validation set (using a method called [k-folds](http://scikit-learn.org/stable/modules/cross_validation.html#k-fold)) and then we train and evaluate models with different hyperparameters selected from a grid of combinations.

In [ ]:
from sklearn.model_selection import GridSearchCV

dtc = DecisionTreeClassifier()

grid = GridSearchCV(dtc,
                   {'max_depth' : range(1,15),
                   'min_samples_split': range(10, 110, 10)},
                    cv=5,
                    n_jobs=5,
                    scoring='f1',
                   verbose=1)
grid.fit(X, y)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
pd.DataFrame(grid.cv_results_).shape

In [ ]:
len(range(1,15)) * len(range(10, 110, 10))

##### Exercise 3:
1. Find the optimal decision tree classifier using a parameter search of:
    - **max_depth**: 5 to 25, increments of 5
    - **min_samples_split**: \[2, 6, 10 \]
    - **max_features**: *see documentation for this**
    - **random_state**: set seed to be 55

## *Caution*: Compounded growth of hyperparameter grid search

Caution is advised when doing hyperparameter tuning using GridSearchCV. Each additional hyperparameter added to the grid search compoundedly grows the number of parameters to search. For example, consider 3 hyperparameters for decision tree, with respective search arrays

|Hyperparameter| Search Array | Number of hyperparameters|
|---|---|---|
|max_depth |*\[5, 15, 25, 30, 45, 55, 75, 100, 125, 150\]* | 10 |
|max_features |*\['auto','sqrt','log'\]* | 3 |
|min_samples_leaf| *\[25, 50, 75, 100, 125, 150, 175, 200 \]* |8|

Assume that we are doing 5-fold cv on the data,

Total number of hyperparameters to be searched are: $ 10*3*8*5 = 1200 $

If we add 2 more hyperparamters to *max_depth*, we now need to search through $6*3*8*5 = 1440$

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import IntSlider, interact

def plot3DSearchGrid(angle):
    xx = [5, 15, 25, 30, 45, 55, 75, 100, 125, 150]
    yy = ['auto','sqrt','log']
    zz = [25, 50, 75, 100, 125, 150, 175, 200 ]

    X, Y, Z = np.mgrid[range(len(xx)), range(len(yy)), range(25,225,25)]
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111, projection='3d')
    scat = ax.scatter(X, Y, Z, c=Z.flatten(), s=100)
    plt.xticks(np.arange(len(xx)), xx)
    ax.set_xlabel('max_depth', labelpad=30)
    plt.yticks(np.arange(len(yy)), yy)
    ax.set_ylabel('min_samples_leaf', labelpad=30)

    ax.set_zlabel('max_features', labelpad=30)
    ax.view_init(30, angle)
    
axes_angle = IntSlider(min=0, max=180, step=15, description='Angle') 
interact(plot3DSearchGrid, angle=axes_angle);

# Further look at Decision Trees and other Tree Based Models
## Random forests

<img src ="00_pics/random_forest.png" >

The performance of a single decision tree will be limited. Instead of relying on one tree, a better approach is to aggregate the predictions of multiple tree. On average, aggregation will perform better than a single predictor. You can envision the aggregation as mimicking the idea of "wisdom of the crowd". We call a tree based model that aggregates the predictions of multiple trees a __random forest__.

In order for a random forest to be effective, the model needs a diverse collection of trees. There should be variations in the chosen thresholds for splitting and the number of nodes and branches. There is no point in aggregating the predicted results if all the trees are nearly identical and produce the same result. There is no "wisdom of the crowd" if everyone thinks alike. To achieve a diverse set of trees, we need to:

1. Train each tree in the forest using a different training set.
1. Only consider a subset of features when deciding how to split the nodes.

For the first point, ideally we would generate a new training set for each tree. However, often times it's too difficult or expensive to collect more data; we have to make due with what we have. Bootstrapping is a general statistical technique to generate "new" data sets with a single set by random sampling with _replacement_. Sampling with replacement allows for a data point to be sampled more than once.

Typically, when training the standard decision tree model, the algorithm will consider all features in deciding the node split. Considering only a subset of your features ensures that your trees do not resemble each other. If the algorithm had considered all features, a dominant feature would be continuously chosen for node splits.

The hyperparameters available for random forests include those of decision tress with some additions.

|Hyperparameter | Description |
|---|---|
|n_estimators|The number of trees in the forest|
|n_jobs|The number of jobs to run in parallel when fitting and predicting|
|warm_start|If set to `True`, reuse the trained tree from a prior fitting and just train the additional trees|


Since the random forest is based on idea of bootstrapping and aggregating the results, it is referred to as a *bagging* ensemble model.

Let's construct a plot of the test set f1-Score as a function of the size of the forest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# X, y = make_moons(n_samples=10000, noise=0.3, random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

def rfc_f1Score(max_features,n_max):
    rfc = RandomForestClassifier(max_features=max_features,
                                max_depth=8,
                                n_estimators=1,
                                warm_start=True,
                                random_state=0)
    
    f1Score = np.zeros(n_max)

    for n in range(1, n_max):
        rfc.set_params(n_estimators=n)
        rfc.fit(X_train, y_train)
        f1Score[n-1] = f1_score(y_test, rfc.predict(X_test))
    
    return f1Score

f1 = rfc_f1Score(max_features='sqrt', n_max=100)
plt.plot(f1[:-1], label = 'sqrt')
plt.xlabel('Number of Trees')
plt.ylabel('f1-Score');
# plt.legend();    

There are several takeaways from the results plotted above.

1. There is sharp increase in metric performance when initially growing the forest.
1. In general, as the number trees increase, performance increases.
1. You _can_ overfit with a large number of trees but the model is robust to overfitting with the number of trees
1. Note, the model was not tuned for hyperparameters like `max_depth` and `min_samples_split`.

The initial increase in f1-Score can be attributed to a large increase in diverse trees when the forest is small. In other words, the additional trees will be very different from the current trees simply because the forest is small. The increase in tree diversity drives predictive power. As the forest grows, newer trees will not be significantly different from the current pool of trees as bootstrapping is no longer producing substantially diverse training sets to create very different looking trees.

# Exercise 4:
1. Fit a Random Forest Classifier and evaluate the f1-score
2. Compare to the default Decision Tree result

## Gradient Boosting Trees
<img src="00_pics/gradient_boosting.png" >

Gradient boosting trees are another ensemble model; it is collection of tree models arranged in a sequence. Here, the model is built stage-wise; each additional tree aims to correct the previously built model's predictions. A model with $M$ trees is equal to 

$$ f_M(x_j) =  \sum^M_m \gamma_m h_m(x_j), $$

where $h_m$ is a **weak learner** decision tree, a "stump" model with low depth that performs poorly on its own. The term **boosting** refers to the algorithm's ability to combine multiple weak learners to form a strong learner. $\gamma_m$ is a factor that scales the contribution of a tree to the overall model. How are $h_m$ and $\gamma_m$ chosen? The model is usually initialized with $h_0$ being equal to the mean of the training labels for regression or the majority class for classification. We also need to choose a loss function $L(y, f_m)$. For example, if the loss function is squared error, then

$$ L_{SE}(y_j, f_m(x_j)) = (y_j-f_m(x_j))^2. $$ 

The steps for building our model with $M$ trees at each stage is

1. Compute the **pseudo-residuals**, the derivative of the loss function with respect to the previous model $f_{m-1}$. The equation for the pseudo-residuals for a model at stage $m$ is
$$ r_{jm} = - \left[\frac{\partial L(y_j, f(x_j))}{\partial f} \right]_{f(x_j) = f_{m-1}(x_j)}. $$
1. Train $h_m$ on the **pseudo-residuals** $r_{jm}$.
1. Choose $\gamma_m$ that minimizes $L(y_j, f_{m-1}(x_j) + \gamma_m h_m(x_j)).$
1. Form the improved model equal to
$$ f_m(x_j) = f_{m-1}(x_j) + \gamma_m h_m(x_j). $$
1. Repeat until the model includes all $M$ trees.

Where does the name gradient come from in gradient boosting trees? Adding a model is analogous to a single iteration in gradient descent. Gradient descent is a minimization algorithm that updates/improves the current answer by taking a step in the direction of the negative of the gradient of the function that is being minimized. The pseudo-residuals represent the direction of greatest reduction in prediction error. Since $h_m$ is trained on the direction of greatest descent of the loss, it will be an approximation of the improvement required for our model to fit the data more closely. $\gamma_m$ is the step size we should take in the direction of greatest model improvement. Compare the equation above for an improved model with the equation of gradient descent applied to a function $C(\beta_i)$;

$$\beta^{updated}_i = \beta^{current}_i - \gamma \left(\left.\frac{\partial C}{\partial \beta_i}\right)\right|_{\beta_i=\beta^{current}_i}.$$

The concept of pseudo-residuals allows us to generalize gradient boosting trees for any loss function. In fact, when we use a square loss function like the squared error, the pseudo-residual is directly proportional to the residual, $y_j - f(x_j)$

Gradient boosting trees have a similar set of hyperparameters as random forests but with some key additions.

|Hyperparameter | Description |
|---|---|
|learning_rate|Multiplicative factor of the tree's contribution to the model|
|subsample|Fraction of the training data to use when fitting the trees|

The learning rate ranges form 0 to 1, with the modified equation being

$$ f_m(x_j) = f_{m-1}(x_j) + \nu \gamma_m h_m(x_j), $$

where $\nu$ is the learning rate. The learning rate and subsampling fraction interact with each other, we will see this in the figure below.

# Exercise 5:
1. Fit a Gradient Tree Classifier, evaluate the f1-score
2. Compare against the previous 2 models
3. What can you do to improve your result?

# Summary

So we now know how to:
- Obtain the optimal hyperparameter for our model using GridSeachCV (hyperparameter tuning)
- Modelling techniques:
    - *Decistion Tree Classifier*
    - Random Forest Classifiers, 
    - Gradient Boosted Tree Classifiers. 